In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

import findspark
findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("my_first_pyspark")\
        .config('spark.ui.port','4050')\
        .getOrCreate()

sc = spark.sparkContext

Mounted at /content/drive


In [ ]:
df=spark.read.csv('/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/data-final.csv', sep ='\t', header = True)
df = df.filter((df.country=='KR') | (df.country=='US') )
df.show()

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+------+-------+-------------------+-------+-------+-----------+----------+---------+---+-------+--------------------+---------------------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|EXT1_E|E

In [ ]:

aaa = df.columns[:50]
aaa.append('country')

df_selected = df.select(aaa)
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
df_selected = df_selected.select([F.col(c).cast('int') if c in df.columns[:50] else c for c in df_selected.columns])
df_selected.show()

+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|EST1|EST2|EST3|EST4|EST5|EST6|EST7|EST8|EST9|EST10|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|country|
+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+-----+-------+
|   4|   3|   4|   3|   3|   3|   5|   3|   4|    3|   2|   4|   4|   2|   4|   2|   2|   2|   4|    4|   1|   2|   1|   5|   3|   5|   3|   4|   4|    5|   3|   2|   4|   2|   1|   4|   4|   2|   2|    5|

In [ ]:
df_selected.count()

547996

In [ ]:
df_selected = df_selected.dropna(how='any')
df_selected.count()

547505

# 1.Dimension reduction (PCA)
Params
- k: 찾고자하는principal components 개수
- inputCol:입력Vectors 컬럼
- outputCol:출력Vectors 컬럼(projection된k-dimensional vector

In [ ]:

import numpy as np
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt 
from pyspark.ml.linalg import Vectors

cols = df_selected.columns[:-1]

# VectorAssembler를 이용해 dataframe의 row를 vector로 변환 
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = cols, outputCol = "feature_vector") 

df_selected = assembler.transform(df_selected)
df_selected = df_selected.select("feature_vector","country")
df_selected.show()
df_selected.take(1)

+--------------------+-------+
|      feature_vector|country|
+--------------------+-------+
|[4.0,3.0,4.0,3.0,...|     US|
|[3.0,2.0,2.0,4.0,...|     US|
|[1.0,2.0,3.0,4.0,...|     US|
|[3.0,4.0,1.0,4.0,...|     US|
|[3.0,2.0,5.0,3.0,...|     US|
|[3.0,1.0,5.0,4.0,...|     US|
|[4.0,3.0,1.0,3.0,...|     US|
|[2.0,4.0,3.0,4.0,...|     US|
|[3.0,1.0,4.0,2.0,...|     US|
|[4.0,2.0,4.0,2.0,...|     US|
|[3.0,2.0,4.0,2.0,...|     US|
|[2.0,2.0,4.0,4.0,...|     US|
|[2.0,1.0,4.0,4.0,...|     US|
|[3.0,2.0,4.0,2.0,...|     US|
|[2.0,5.0,1.0,5.0,...|     US|
|[1.0,1.0,3.0,4.0,...|     US|
|[4.0,1.0,3.0,1.0,...|     US|
|[3.0,3.0,4.0,4.0,...|     US|
|[4.0,1.0,5.0,1.0,...|     US|
|[5.0,1.0,3.0,2.0,...|     US|
+--------------------+-------+
only showing top 20 rows



[Row(feature_vector=DenseVector([4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 5.0, 3.0, 4.0, 3.0, 2.0, 4.0, 4.0, 2.0, 4.0, 2.0, 2.0, 2.0, 4.0, 4.0, 1.0, 2.0, 1.0, 5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 5.0, 3.0, 2.0, 4.0, 2.0, 1.0, 4.0, 4.0, 2.0, 2.0, 5.0, 5.0, 2.0, 4.0, 3.0, 4.0, 1.0, 5.0, 5.0, 4.0, 4.0]), country='US')]

In [ ]:
#df_selected= df_selected.limit(10000)
pca = PCA(k=10, inputCol = "feature_vector", outputCol ="features")
model =pca.fit(df_selected)
reduced = model.transform(df_selected)
reduced.show()

+--------------------+-------+--------------------+
|      feature_vector|country|            features|
+--------------------+-------+--------------------+
|[4.0,3.0,4.0,3.0,...|     US|[-0.0404338023452...|
|[3.0,2.0,2.0,4.0,...|     US|[3.22162795473366...|
|[1.0,2.0,3.0,4.0,...|     US|[4.69501703285355...|
|[3.0,4.0,1.0,4.0,...|     US|[6.10108803275653...|
|[3.0,2.0,5.0,3.0,...|     US|[-2.3186438995364...|
|[3.0,1.0,5.0,4.0,...|     US|[-2.8277269804500...|
|[4.0,3.0,1.0,3.0,...|     US|[6.31746947321205...|
|[2.0,4.0,3.0,4.0,...|     US|[5.84919408718216...|
|[3.0,1.0,4.0,2.0,...|     US|[-2.5611757128723...|
|[4.0,2.0,4.0,2.0,...|     US|[-2.7292970198383...|
|[3.0,2.0,4.0,2.0,...|     US|[-3.0511732190363...|
|[2.0,2.0,4.0,4.0,...|     US|[2.72047781931732...|
|[2.0,1.0,4.0,4.0,...|     US|[3.09442246076775...|
|[3.0,2.0,4.0,2.0,...|     US|[0.28542967330603...|
|[2.0,5.0,1.0,5.0,...|     US|[9.17789858715705...|
|[1.0,1.0,3.0,4.0,...|     US|[4.42452558913240...|
|[4.0,1.0,3.

#2.Clustering


## gaussian mix


In [ ]:
from pyspark.ml.clustering import GaussianMixture

gm = GaussianMixture(featuresCol ="features",k=2, tol = 0.0001,seed =10)
model =gm.fit(reduced)
prob = model.transform(reduced)
prob.show()
prob.take(1)

+--------------------+-------+--------------------+--------------------+----------+
|      feature_vector|country|            features|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[4.0,3.0,4.0,3.0,...|     US|[-0.0404338023452...|[0.00414997473876...|         1|
|[3.0,2.0,2.0,4.0,...|     US|[3.22162795473366...|[0.55210984750194...|         0|
|[1.0,2.0,3.0,4.0,...|     US|[4.69501703285355...|[0.06254021524333...|         1|
|[3.0,4.0,1.0,4.0,...|     US|[6.10108803275653...|[0.98213096685105...|         0|
|[3.0,2.0,5.0,3.0,...|     US|[-2.3186438995364...|[4.22488181491523...|         1|
|[3.0,1.0,5.0,4.0,...|     US|[-2.8277269804500...|[1.49277956452890...|         1|
|[4.0,3.0,1.0,3.0,...|     US|[6.31746947321205...|[0.57733692357984...|         0|
|[2.0,4.0,3.0,4.0,...|     US|[5.84919408718216...|[0.03649563136597...|         1|
|[3.0,1.0,4.0,2.0,...|     US|[-2.5611757128723...|[1.38278229375290...|    

[Row(feature_vector=DenseVector([4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 5.0, 3.0, 4.0, 3.0, 2.0, 4.0, 4.0, 2.0, 4.0, 2.0, 2.0, 2.0, 4.0, 4.0, 1.0, 2.0, 1.0, 5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 5.0, 3.0, 2.0, 4.0, 2.0, 1.0, 4.0, 4.0, 2.0, 2.0, 5.0, 5.0, 2.0, 4.0, 3.0, 4.0, 1.0, 5.0, 5.0, 4.0, 4.0]), country='US', features=DenseVector([-0.0404, -10.0504, -8.7572, 5.7228, 7.8789, -16.6939, 0.1438, 3.3011, -0.676, 2.187]), probability=DenseVector([0.0041, 0.9959]), prediction=1)]

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCols=['country'], outputCols=['country_index'])
prob = indexer.fit(prob).transform(prob)

ohe_encoder = OneHotEncoder(inputCols=['country_index'], outputCols=['country_ohe'],dropLast=False)
encoding_df = ohe_encoder.fit(prob).transform(prob)

encoding_df.limit(5).show()

+--------------------+-------+--------------------+--------------------+----------+-------------+-------------+
|      feature_vector|country|            features|         probability|prediction|country_index|  country_ohe|
+--------------------+-------+--------------------+--------------------+----------+-------------+-------------+
|[4.0,3.0,4.0,3.0,...|     US|[-0.0404338023452...|[0.00414997473876...|         1|          0.0|(2,[0],[1.0])|
|[3.0,2.0,2.0,4.0,...|     US|[3.22162795473366...|[0.55210984750194...|         0|          0.0|(2,[0],[1.0])|
|[1.0,2.0,3.0,4.0,...|     US|[4.69501703285355...|[0.06254021524333...|         1|          0.0|(2,[0],[1.0])|
|[3.0,4.0,1.0,4.0,...|     US|[6.10108803275653...|[0.98213096685105...|         0|          0.0|(2,[0],[1.0])|
|[3.0,2.0,5.0,3.0,...|     US|[-2.3186438995364...|[4.22488181491523...|         1|          0.0|(2,[0],[1.0])|
+--------------------+-------+--------------------+--------------------+----------+-------------+-------

In [ ]:
df_col_onehot=encoding_df

#3. (quiz) Correlation between Clusters and Country

Given
- Big five personality test records
  - where country is KR or US
- Cluster result with KMeansand GaussianMixture▪

Goal

- For each clustering method, analyze the correlation between country attribute and the probability distributions that a person belongs to Group 1 or 2

In [ ]:
df = df_col_onehot.select("probability", "country_ohe")
df.show()

+--------------------+-------------+
|         probability|  country_ohe|
+--------------------+-------------+
|[0.00414997473876...|(2,[0],[1.0])|
|[0.55210984750194...|(2,[0],[1.0])|
|[0.06254021524333...|(2,[0],[1.0])|
|[0.98213096685105...|(2,[0],[1.0])|
|[4.22488181491523...|(2,[0],[1.0])|
|[1.49277956452890...|(2,[0],[1.0])|
|[0.57733692357984...|(2,[0],[1.0])|
|[0.03649563136597...|(2,[0],[1.0])|
|[1.38278229375290...|(2,[0],[1.0])|
|[0.00230862630696...|(2,[0],[1.0])|
|[6.73315319754263...|(2,[0],[1.0])|
|[0.01550209071294...|(2,[0],[1.0])|
|[0.00267612094088...|(2,[0],[1.0])|
|[6.92194620669427...|(2,[0],[1.0])|
|[0.01318631612595...|(2,[0],[1.0])|
|[0.05340729366160...|(2,[0],[1.0])|
|[0.00759148778219...|(2,[0],[1.0])|
|[6.89033817855521...|(2,[0],[1.0])|
|[0.12934146506953...|(2,[0],[1.0])|
|[0.32451904123662...|(2,[0],[1.0])|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
ddf = df
assembler = VectorAssembler(inputCols = ["probability","country_ohe"], outputCol = "cor_input")
ddf= assembler.transform(ddf)
ddf.show()

+--------------------+-------------+--------------------+
|         probability|  country_ohe|           cor_input|
+--------------------+-------------+--------------------+
|[0.00414997473876...|(2,[0],[1.0])|[0.00414997473876...|
|[0.55210984750194...|(2,[0],[1.0])|[0.55210984750194...|
|[0.06254021524333...|(2,[0],[1.0])|[0.06254021524333...|
|[0.98213096685105...|(2,[0],[1.0])|[0.98213096685105...|
|[4.22488181491523...|(2,[0],[1.0])|[4.22488181491523...|
|[1.49277956452890...|(2,[0],[1.0])|[1.49277956452890...|
|[0.57733692357984...|(2,[0],[1.0])|[0.57733692357984...|
|[0.03649563136597...|(2,[0],[1.0])|[0.03649563136597...|
|[1.38278229375290...|(2,[0],[1.0])|[1.38278229375290...|
|[0.00230862630696...|(2,[0],[1.0])|[0.00230862630696...|
|[6.73315319754263...|(2,[0],[1.0])|[6.73315319754263...|
|[0.01550209071294...|(2,[0],[1.0])|[0.01550209071294...|
|[0.00267612094088...|(2,[0],[1.0])|[0.00267612094088...|
|[6.92194620669427...|(2,[0],[1.0])|[6.92194620669427...|
|[0.0131863161

In [ ]:
df = ddf.select("cor_input")
df.show(truncate = False)

+--------------------------------------------------+
|cor_input                                         |
+--------------------------------------------------+
|[0.004149974738769418,0.9958500252612307,1.0,0.0] |
|[0.5521098475019427,0.44789015249805725,1.0,0.0]  |
|[0.06254021524333596,0.9374597847566641,1.0,0.0]  |
|[0.9821309668510534,0.017869033148946705,1.0,0.0] |
|[4.2248818149152325E-4,0.9995775118185085,1.0,0.0]|
|[1.4927795645289054E-4,0.9998507220435472,1.0,0.0]|
|[0.577336923579844,0.422663076420156,1.0,0.0]     |
|[0.03649563136597397,0.9635043686340261,1.0,0.0]  |
|[1.382782293752907E-4,0.9998617217706247,1.0,0.0] |
|[0.002308626306961465,0.9976913736930385,1.0,0.0] |
|[6.733153197542636E-4,0.9993266846802458,1.0,0.0] |
|[0.015502090712943659,0.9844979092870563,1.0,0.0] |
|[0.002676120940889509,0.9973238790591105,1.0,0.0] |
|[6.921946206694279E-4,0.9993078053793306,1.0,0.0] |
|[0.01318631612595749,0.9868136838740426,1.0,0.0]  |
|[0.053407293661603204,0.9465927063383969,1.0,

In [ ]:
from pyspark.ml.stat import Correlation

Correlation.corr(df,'cor_input','pearson').take(1)

[Row(pearson(cor_input)=DenseMatrix(4, 4, [1.0, -1.0, -0.0117, 0.0117, -1.0, 1.0, 0.0117, -0.0117, -0.0117, 0.0117, 1.0, -1.0, 0.0117, -0.0117, -1.0, 1.0], False))]